In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import scipy.stats
from scipy.stats import norm
from sklearn.linear_model import LogisticRegression
from tqdm.auto import tqdm
import time
import scipy.linalg as scilinalg
import seaborn as sns
from scipy.stats import ortho_group
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

import multiprocessing as mp
from joblib import Parallel, delayed
from utils import *

from conf_simu import *

np.random.seed(1234)

/Users/yugui/opt/anaconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def simu_hetero(d1,d2,alpha,het,sd,tail,pr,k_star,rk,M_mean,mis_set,full_exp=False):
    M_star, A, P, S = gen_data(d1,d2,het,sd,tail,pr,M_mean,mis_set,k_star)
    coverage_cmc_als, coverage_cmc_als_hat, length_cmc_als, length_cmc_als_hat = cfmc_simu_hetero(M_star,S,P,"als",het,rk,alpha)
    if full_exp:
        coverage_cmc_cvx, coverage_cmc_cvx_hat, length_cmc_cvx, length_cmc_cvx_hat = cfmc_simu_hetero(M_star,S,P,"cvx",het,rk,alpha)
        return coverage_cmc_cvx, coverage_cmc_als, coverage_cmc_cvx_hat, coverage_cmc_als_hat, length_cmc_cvx, length_cmc_als, length_cmc_cvx_hat, length_cmc_als_hat
    else:
        return coverage_cmc_als, coverage_cmc_als_hat, length_cmc_als, length_cmc_als_hat

In [5]:
d1 = d2 = 500
alpha = 0.1
sd = 1
het = 'logis2' # 'homo' | 'logis' | 'rank1'
pr = 0.8
rk = 8
M_mean = 5
mis_set = 0
k_star = 8
full_exp = True
tail = 'gaussian'
if full_exp:
    coverage_cmc_cvx, coverage_cmc_als, coverage_cmc_cvx_hat, coverage_cmc_als_hat, length_cmc_cvx, length_cmc_als, length_cmc_cvx_hat, length_cmc_als_hat = simu_hetero(d1,d2,alpha,het,sd,tail,pr,k_star,rk,M_mean,mis_set,full_exp=full_exp)
    print('coverage rate: \n\t cmc*-als {}\n\t cmc*-cvx {}\n\t cmc-als {}\n\t cmc-cvx {}'.format(coverage_cmc_als, coverage_cmc_cvx, coverage_cmc_als_hat, coverage_cmc_cvx_hat))
    print('Average length: \n\t cmc*-als {}\n\t cmc*-cvx {}\n\t cmc-als {}\n\t cmc-cvx {}'.format(length_cmc_als, length_cmc_cvx, length_cmc_als_hat, length_cmc_cvx_hat))
else:
    coverage_cmc_als, coverage_cmc_als_hat, length_cmc_als, length_cmc_als_hat = simu_hetero(d1,d2,alpha,het,sd,tail,pr,k_star,rk,M_mean,mis_set,full_exp=full_exp)
    print('coverage rate: \n\t cmc*-als {}\n\t cmc-als {}'.format(coverage_cmc_als, coverage_cmc_als_hat))
    print('Average length: \n\t cmc*-als {}\n\t cmc-als {}'.format(length_cmc_als, length_cmc_als_hat))

iter: 1
iter: 2
iter: 3
iter: 4
iter: 5
iter: 6
iter: 7
Function value changing by less than progTol
iter: 1
iter: 2
iter: 3
iter: 4
iter: 5
iter: 6
iter: 7
Function value changing by less than progTol
coverage rate: 
	 cmc*-als 0.8978621758512151
	 cmc*-cvx 0.8985811750043938
	 cmc-als 0.8995797849393644
	 cmc-cvx 0.8998753734801157
Average length: 
	 cmc*-als 3.4127
	 cmc*-cvx 3.4235
	 cmc-als 3.4364
	 cmc-cvx 3.4313


In [ ]:
# repeated
# remember to disable plotting
alpha = 0.1
sd0 = 1
base1 = 'cvx'
base2 = base = 'als'
repN = 100
pr = 0.8
M_mean = 2
k_star = 8
tail = 'gaussian'
sigma_true=False
full_exp=False
num_cores = mp.cpu_count()


for d in [500]:
    d1 = d2 = d
    for full_exp in [True]:
        if full_exp:
            rk_seq = range(4,25,4)
        else:
            rk_seq = range(2,41,2)
        for het in ['logis2','logis1']:
            for mis_set in [4,5,0]:
                for rk in rk_seq:
                    if mis_set==4:
                        tail='het'
                        sd = sd0
                    elif mis_set==5:
                        tail='het1'
                        sd=sd0
                    else:
                        tail='gaussian'
                        sd = sd0
                    print([mis_set, tail, sd])
                    if __name__ == "__main__":
                        results = Parallel(n_jobs=num_cores)(delayed(simu_hetero)(d1,d2,alpha,het,sd,tail,pr,k_star,rk,M_mean,mis_set,full_exp=full_exp) for i in range(repN))
                    results = np.array(results)

                    if full_exp:
                        res_mat = results.reshape(repN,8)

                        cov_rt_cvx = res_mat[:,0]
                        cov_rt_als = res_mat[:,1]
                        cov_rt_cvx_hat = res_mat[:,2]
                        cov_rt_als_hat = res_mat[:,3]
                        cov_ = np.hstack((cov_rt_cvx, cov_rt_als))
                        cov_ = np.hstack((cov_,cov_rt_cvx_hat))
                        cov_ = np.hstack((cov_,cov_rt_als_hat))
                        len_ave_cvx = res_mat[:,4]
                        len_ave_als = res_mat[:,5]
                        len_ave_cvx_hat = res_mat[:,6]
                        len_ave_als_hat = res_mat[:,7]
                        len_ = np.hstack((len_ave_cvx, len_ave_als))
                        len_ = np.hstack((len_,len_ave_cvx_hat))
                        len_ = np.hstack((len_,len_ave_als_hat))

                        label1 = 'cf*-'+base1
                        label2 = 'cf*-'+base2
                        label3 = 'cf-'+base1
                        label4 = 'cf-'+base2
                        nam_ = [label1]*repN + [label2]*repN + [label3]*repN + [label4]*repN
                        cov_df = pd.DataFrame(cov_, columns=['coverage'])
                        len_df = pd.DataFrame(len_, columns=['length'])
                        cov_df['approach'] = nam_
                        len_df['approach'] = nam_

                    else:
                        res_mat = results.reshape(repN,4)

                        cov_rt_als = res_mat[:,0]
                        cov_rt_als_hat = res_mat[:,1]
                        cov_ = np.hstack((cov_rt_als, cov_rt_als_hat))
                        len_ave_als = res_mat[:,2]
                        len_ave_als_hat = res_mat[:,3]
                        len_ = np.hstack((len_ave_als, len_ave_als_hat))

                        label1 = 'cf*-'+base2
                        label2 = 'cf-'+base2
                        nam_ = [label1]*repN + [label2]*repN
                        cov_df = pd.DataFrame(cov_, columns=['coverage'])
                        len_df = pd.DataFrame(len_, columns=['length'])
                        cov_df['approach'] = nam_
                        len_df['approach'] = nam_

                    file_dir1 = '../results/hetero_cov_'+str(d1)+'_'+str(mis_set)+'_'+str(k_star)+'_'+str(rk)+'_'+het+'_'+str(sd)+'_'+tail+'_'+str(full_exp)+'.csv'
                    file_dir2 = '../results/hetero_len_'+str(d1)+'_'+str(mis_set)+'_'+str(k_star)+'_'+str(rk)+'_'+het+'_'+str(sd)+'_'+tail+'_'+str(full_exp)+'.csv'
                    cov_df.to_csv(file_dir1, index=False)
                    len_df.to_csv(file_dir2, index=False)
